In [1]:
import itertools
import gurobipy as gpy
import numpy as np

# File path for the distance data
file_path = "C:\\Users\\zhihu\\Desktop\\Optimization (Fall 2023)\\MST Assignment\\mst_data\\dist_5.txt"

# Reading the file and converting the distances into a list of floats
with open(file_path, 'r') as file:
    distances = [float(line.strip()) for line in file.readlines()]

# Number of locations
n = 5

# Create an empty n x n distance matrix
distance_matrix = np.zeros((n, n))

# Fill the upper triangular part of the matrix with the distance data
distance_matrix[np.triu_indices(n, 1)] = distances

# Make the matrix symmetric
distance_matrix = distance_matrix + distance_matrix.T

# Define locations
locs = range(n)

# Construct Gurobi model
m = gpy.Model('mst')
x = {}
for i in range(n):
    for j in range(i+1, n):
        x[i, j] = m.addVar(vtype=gpy.GRB.BINARY, name=f'x_{i}_{j}')

m.update()
m.setObjective(gpy.quicksum(x[i, j] * distance_matrix[i, j] for i in range(n) for j in range(i+1, n)), gpy.GRB.MINIMIZE)
m.addConstr(gpy.quicksum(x.values()) == n - 1)
m.update()
m.optimize()

# Print initial solution
for v in m.getVars():
    print(f'{v.varName}: {v.x}')

# Subtour elimination
for mm in range(2, n):
    subsets = itertools.combinations(locs, mm)
    for subset in subsets:
        num_links = sum([x[i, j].x for i, j in itertools.combinations(subset, 2) if (i, j) in x])
        if num_links >= len(subset):
            m.addConstr(gpy.quicksum(x[i, j] for i, j in itertools.combinations(subset, 2) if (i, j) in x) <= len(subset) - 1)

# Update and re-optimize
m.update()
m.optimize()

# Print final solution
for v in m.getVars():
    if v.x > 0.5:  # Selecting edges that are part of the final solution
        print(f'{v.varName}: {v.x}')


Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-06
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12800H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 1 rows, 10 columns and 10 nonzeros
Model fingerprint: 0xd65a2546
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 4e+00]
Found heuristic solution: objective 182.0000000
Presolve removed 1 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 2: 126.5 182 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.265000000000e+02, best bound